# 003 - Window Functions

```SQL
DROP TABLE IF EXISTS ventas_electronica;

CREATE TABLE ventas_electronica (
    id_venta INT,
    fecha DATE,
    producto VARCHAR(50),
    categoria VARCHAR(50),
    precio DECIMAL(10,2),
    vendedor_id INT
);

INSERT INTO ventas_electronica VALUES
(1, '2023-11-01', 'iPhone 15', 'Celulares', 1200, 10),
(2, '2023-11-01', 'Samsung S23', 'Celulares', 900, 11),
(3, '2023-11-02', 'MacBook Air', 'Laptops', 1100, 10),
(4, '2023-11-02', 'Dell XPS', 'Laptops', 1300, 12),
(5, '2023-11-03', 'iPhone 15', 'Celulares', 1200, 11),
(6, '2023-11-03', 'iPad Pro', 'Tablets', 800, 10),
(7, '2023-11-04', 'Samsung S23', 'Celulares', 900, 12),
(8, '2023-11-04', 'MacBook Pro', 'Laptops', 2500, 10),
(9, '2023-11-05', 'Lenovo Yoga', 'Laptops', 950, 11),
(10, '2023-11-05', 'iPad Air', 'Tablets', 600, 12);

SELECT * FROM ventas_electronica;
```

In [3]:
import pandas as pd
import numpy as np

In [7]:
df = pd.read_csv('DATA/003/ventas_electronica.csv', parse_dates=['fecha'])

df.head()

,id_venta,fecha,producto,categoria,precio,vendedor_id
0,1,2023-11-01,iPhone 15,Celulares,1200.0,10
1,2,2023-11-01,Samsung S23,Celulares,900.0,11
2,3,2023-11-02,MacBook Air,Laptops,1100.0,10
3,4,2023-11-02,Dell XPS,Laptops,1300.0,12
4,5,2023-11-03,iPhone 15,Celulares,1200.0,11


# Ejercicio 1: Totales Relativos (Nivel Básico)

**Objetivo: Comparar cada venta individual con el total de su categoría.**

- El Reto: Escribe una consulta que muestre: producto, categoria, precio y una columna llamada total_categoria que sume los precios pero particionados por categoría.

- Pista: Usa SUM(precio) OVER(PARTITION BY categoria).

In [8]:
df['sum_categoria'] = df.groupby('categoria')['precio'].transform('sum')

respuesta= df[['producto','categoria','sum_categoria']]

respuesta

,producto,categoria,sum_categoria
0,iPhone 15,Celulares,4200.0
1,Samsung S23,Celulares,4200.0
2,MacBook Air,Laptops,5850.0
3,Dell XPS,Laptops,5850.0
4,iPhone 15,Celulares,4200.0
5,iPad Pro,Tablets,1400.0
6,Samsung S23,Celulares,4200.0
7,MacBook Pro,Laptops,5850.0
8,Lenovo Yoga,Laptops,5850.0
9,iPad Air,Tablets,1400.0


```SQL
SELECT
    producto,
    categoria,
    SUM(precio) OVER(PARTITION BY categoria) as sum_categoria
FROM ventas_electronica
```

In [9]:
# Ordenamos primero para que el ranking tenga sentido visual
df = df.sort_values(['categoria', 'precio'], ascending=[True, False])

# 1. ROW_NUMBER -> method='first'
df['fila_n'] = df.groupby('categoria')['precio'].rank(method='first', ascending=False)

# 2. RANK -> method='min' (deja huecos si hay empates: 1, 1, 3)
df['rango'] = df.groupby('categoria')['precio'].rank(method='min', ascending=False)

# 3. DENSE_RANK -> method='dense' (no deja huecos: 1, 1, 2)
df['rango_denso'] = df.groupby('categoria')['precio'].rank(method='dense', ascending=False)